In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

print("Pandas Version: " + pd.__version__)
print("Numpy Version: " + np.__version__)

Pandas Version: 1.2.0
Numpy Version: 1.18.5


In [2]:
trips = pd.read_csv("../data/clean/trips.csv", parse_dates=["date_from", "date_until"])
trips.head(3)

,bike_id,user_id,date_from,date_until,start_station_name,start_station_id,end_station_name,end_station_id,booked_via,duration_in_min,distance_in_km
0,143517,A821059B555C7764A2FF801180874A2FCB326222,2014-01-01 00:34:54,2014-01-01 00:50:14,U-Bahn Baumwall,214170,Mönckebergstraße / Rosenstraße,131880,iPhone SRH,16,1.293661
1,119830,1EBC930DB407ACEAE2FDE23A6CA40492EA3DFBB2,2014-01-01 01:39:55,2014-01-01 01:57:27,Bahnhof Altona Ost/Max-Brauer-Allee,131646,Schulterblatt/Eifflerstraße,131648,Android SRH,18,2.032271
2,143501,7AD2C1B70137479062A6DD73815835986677BB2D,2014-01-01 01:40:20,2014-01-01 01:53:09,Weidestraße/Biedermannplatz,211922,Jarrestraße / Rambatzweg,138376,Techniker HH_119 (-2334-),13,0.954178


In [3]:
weather = pd.read_csv("../data/clean/weather.csv", parse_dates=["datetime"])
weather.head(3)

,datetime,precip_intensity,precip_probability,precip_type,temperature,humidity,wind_speed,wind_bearing,uv_index,visibility
0,2014-01-01 00:00:00,0.0,0.0,none,1.93,0.81,3.91,161.0,0.0,9.988
1,2014-01-01 01:00:00,0.0,0.0,none,1.94,0.85,4.56,140.0,0.0,6.004
2,2014-01-01 02:00:00,0.0,0.0,none,1.81,0.85,4.12,143.0,0.0,9.988


In [4]:
stations = pd.read_csv("../data/clean/stations.csv")
stations.drop(columns = ["index"], inplace=True)
stations.head(3)

,station_id,name,longitude,latitude
0,131543,Landungsbrücke/Hafentor,9.97230,53.5457
1,131546,Fischmarkt/Breite Straße,9.95088,53.5462
2,131547,Paulinenplatz/Wohlwillstraße,9.96246,53.5542


## Accumulate stations per hour and add weather data

TODO: Add number of customers

### Counting check-ins and check-outs per station per hour

In [5]:
trips_slice = trips[["date_from", "date_until", "start_station_name", "end_station_name"]].copy()

In [6]:
check_ins = trips_slice.groupby([pd.Grouper(key='date_until', freq='H'), 'end_station_name']).size().to_frame('check_ins')
check_ins.index.rename(['datetime', 'station_name'], inplace=True)
check_ins.sample(3)

,,check_ins
datetime,station_name,
2016-04-01 19:00:00,Schleidenstraße/Osterbekstraße,10
2014-07-31 14:00:00,Gänsemarkt / Büschstraße,7
2014-02-18 14:00:00,Eiffestrasse/Normannenweg,1


In [7]:
check_outs = trips_slice.groupby([pd.Grouper(freq='H', key='date_from'), 'start_station_name']).size().to_frame('check_outs')
check_outs.index.rename(['datetime', 'station_name'], inplace=True)
check_outs.sample(3)

,,check_outs
datetime,station_name,
2016-02-03 14:00:00,Unilever/Strandkai,3
2015-10-13 10:00:00,Bahrenfelder Straße/Völckersstraße,3
2016-11-11 04:00:00,Kellinghusenstraße / Loogeplatz,1


### Create empty data set with stats per station per hour and fill in accumulated data

In [8]:
# create date range
start_date = datetime(2014, 1, 1, 0)
end_date = datetime(2017, 5, 15, 11, 59, 59)
date_range = pd.date_range(start_date, end_date, freq="H")

station_names = sorted(pd.unique(stations["name"]))

# create multiindex from hourly timestamps and station names
index = pd.MultiIndex.from_product([date_range, station_names], names=['datetime', 'station_name'])

# create empty data set with index
empty_df = pd.DataFrame(index=index)

# insert calculations and fill missing values with 0, meaning no check-outs or check-ins observed
station_count = empty_df.join(check_ins).join(check_outs).fillna(0)
station_count.check_ins = station_count.check_ins.astype(int)
station_count.check_outs = station_count.check_outs.astype(int)

# calculate change in bike inventory
station_count['diff'] = (station_count['check_ins'] - station_count['check_outs'])

station_count.reset_index(inplace=True)

In [9]:
station_trips_weather = pd.merge(station_count, weather, on='datetime', how='left', sort=False)
station_trips_weather.sample(3)

,datetime,station_name,check_ins,check_outs,diff,precip_intensity,precip_probability,precip_type,temperature,humidity,wind_speed,wind_bearing,uv_index,visibility
47522,2014-01-10 12:00:00,Jungfernstieg / Ballindamm,7,1,6,0.0119,0.09,rain,6.84,0.84,7.68,256.0,0.0,10.003
5238462,2016-11-15 08:00:00,Uhlandstraße / Eingang Nord,3,2,1,0.0187,0.07,rain,3.81,0.93,4.81,215.0,0.0,6.004
5424724,2016-12-22 16:00:00,Herthastraße / Einkaufszentrum Bramfeld,0,0,0,0.0000,0.00,none,4.98,0.98,2.48,233.0,0.0,5.409


In [10]:
station_trips_weather.to_csv("../data/clean/station_trips_weather.csv", index=False)

## Add weather data to each trip

In [11]:
# Add 'merge_datetime' as key to merge on

trips["merge_datetime"] = pd.to_datetime(pd.DataFrame(
    {
        "year": pd.DatetimeIndex(trips["date_from"]).year,
        "month": pd.DatetimeIndex(trips["date_from"]).month,
        "day": pd.DatetimeIndex(trips["date_from"]).day,
        "hour": pd.DatetimeIndex(trips["date_from"]).hour
    }
))

In [12]:
# Add weather data to every trip

trips_with_weather = pd.merge(trips, weather, left_on="merge_datetime", right_on="datetime", right_index=False)
trips_with_weather.drop(columns=["merge_datetime", "datetime"], inplace=True)
trips_with_weather.head(3)

,bike_id,user_id,date_from,date_until,start_station_name,start_station_id,end_station_name,end_station_id,booked_via,duration_in_min,distance_in_km,precip_intensity,precip_probability,precip_type,temperature,humidity,wind_speed,wind_bearing,uv_index,visibility
0,143517,A821059B555C7764A2FF801180874A2FCB326222,2014-01-01 00:34:54,2014-01-01 00:50:14,U-Bahn Baumwall,214170,Mönckebergstraße / Rosenstraße,131880,iPhone SRH,16,1.293661,0.0,0.0,none,1.93,0.81,3.91,161.0,0.0,9.988
1,120450,2C6FD52D5611B4DAC29E5197B67979D583685948,2014-01-01 00:27:47,2014-01-01 00:34:41,Lange Reihe / Lohmühlenpark,138385,Hauptbahnhof Ost / Hachmannplatz,131873,IVR,7,0.786254,0.0,0.0,none,1.93,0.81,3.91,161.0,0.0,9.988
2,117503,253528F56A582E05F13D8A89B20A23800DDEBEF2,2014-01-01 00:49:02,2014-01-01 00:56:32,Sternschanze / Eingang Dänenweg,139501,Bahnhof Dammtor Süd / Marseiller Straße,138382,iPhone SRH,8,1.422850,0.0,0.0,none,1.93,0.81,3.91,161.0,0.0,9.988


In [13]:
trips_with_weather.to_csv("../data/clean/trips_with_weather.csv", index=False)